In [39]:
import pandas as pd

# Load the Excel file into a DataFrame
df = pd.read_excel('edited.xlsx', engine='openpyxl')

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Access each column value in the row using the column names
    print(f"Row {index}:")
    for column in df.columns:
        print(f"{column}: {row[column]}")
    print("----")


Row 0:
Unnamed: 0: 0
All_content: ['', '', '', '', '', 'CHIL SEZ IT Park, Saravanampatty, Coimbatore - 035 info@dplus.co.in\nEmployee login\nHOME\nABOUT US\nSERVICES\nFRANCHISE\nCONTACT US\nCALL US NOW\n96 5 97 88888\nWELCOME TO DPLUS CAR CARE\nDplus is a One-Stop Solution for all your car servicing needs.\nFREE ESTIMATE\n01\n02\n03\nWHO IS DPLUS CAR CARE?\nAbout Our Company\nA specialized team of experienced technicians thriving to help clients in their needs.\nDplus Car Care is a leading brand in South India for Professional Car Washing ,Painting and Detailing Services Our services are supported by a team of well trained staffs who comes with a wealth of experience, not only in car washing and valeting professionalism and technical ability but also in customer service. our aim is to deliver the very best for our customers by providing an enhanced experience and an unequalled service.\nOUR SERVICES\nOur Service Plans To Meet All Your Demands\nDplus Car Care is incredibly flexible, fri

In [15]:
import boto3
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from prompt import prompt
client = boto3.client("bedrock-runtime", region_name="us-east-1")
llm = ChatBedrock(client=client,model="mistral.mistral-small-2402-v1:0",temperature=0) #meta.llama3-1-70b-instruct-v1:0 mistral.mixtral-8x7b-instruct-v0:1 mistral.mistral-small-2402-v1:0

In [16]:
#classify

def chat_with_llm(message):
    messages = [SystemMessage(content=prompt), HumanMessage(content=message)]
    response = llm.invoke(messages)
    return response

import re
def extract_json_from_string(s):
    json_pattern = re.compile(r'\{.*\}', re.DOTALL)
    
    # Search for the JSON pattern in the string
    match = json_pattern.search(s)
    
    if match:
        json_str = match.group(0)
        return json_str
    else:
        return None

In [ ]:
import time
import json

import pandas as pd

# Load the Excel file into a DataFrame
df = pd.read_excel('edited.xlsx', engine='openpyxl')


def json_converting(df, coloumn):
    all_data = []
    if coloumn in df.columns:
        pass
    else:
        raise ValueError(f"Column '{coloumn}' not found in the DataFrame.")
    # Loop over your data and call chat_with_llm
    for index, row in df.iterrows():
        print("The index",index)
        try:
            response = chat_with_llm(row[coloumn])
            
            extracted_json_str = extract_json_from_string(response.content)
            
            if extracted_json_str:

                extracted_json = json.loads(extracted_json_str)
                extracted_json["Main_data_Row"] = row["Unnamed: 0"]

                all_data.append(extracted_json)

                print(f"Successfully processed item")

            
        except Exception as e:
            print(f"Error processing item Error: {e}")
        
        # time.sleep(60)  # use sleep whenecer use groq freetrial

    with open("data.json", "w") as json_file:
        json.dump(all_data, json_file, indent=4)

    print("Processing complete. Data stored in data.json.")

In [41]:
json_converting(df, "All_content")

The index 0
Successfully processed item
The index 1
Successfully processed item
The index 2
Successfully processed item
The index 3
Successfully processed item
The index 4
Error processing item Error: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
The index 5
Successfully processed item
The index 6
Successfully processed item
The index 7
Successfully processed item
The index 8
Successfully processed item
The index 9
Successfully processed item
The index 10
Successfully processed item
The index 11
Successfully processed item
The index 12
Successfully processed item
The index 13
Successfully processed item
The index 14
Successfully processed item
The index 15
Successfully processed item
The index 16
Successfully processed item
The index 17
Successfully processed item
The index 18
Error processing item Error: Extra data: line 35 column 1 (char 683)
The index 19
Successfully processed item
The index 20
Error processing item Error: Extra data: line 35 column 1 (

In [ ]:
import openpyxl
import json

def clean_string(value):
    if isinstance(value, str):
        return value.replace("String", "") 
    return value

wb = openpyxl.load_workbook('scraped_2511.xlsx')
sheet = wb.active

with open('data2.json', 'r') as json_file:
    data = json.load(json_file)

header_row = sheet[1]
first_empty_column = len(header_row) + 1

new_headers = [
    "New Business Name",
    "New Address",
    "New Rating",
    "New Working Hours",
    "New Phone Number",
    "New Email",
    "New Services"
]


for i, header in enumerate(new_headers):
    if sheet.cell(row=1, column=first_empty_column + i).value is None:
        sheet.cell(row=1, column=first_empty_column + i, value=header)


for record in data:
    metadata = record.get('Main_data_Row')
    
    if metadata and metadata > 0:  
        row = metadata
        

        sheet.cell(row=row, column=first_empty_column, value=clean_string(record.get('business_name', '')))  # New Business Name
        sheet.cell(row=row, column=first_empty_column + 1, value=clean_string(record.get('location_details', {}).get('full_address', '')))  # New Address
        sheet.cell(row=row, column=first_empty_column + 2, value=clean_string(record.get('ratings_and_reviews', {}).get('average_rating', '')))  # New Rating
        

        working_hours = record.get('operational_details', {}).get('working_hours', {})
        if isinstance(working_hours, dict):
            working_hours_str = "\n".join(f"{day}: {hours}" for day, hours in working_hours.items())
            working_hours_str = clean_string(working_hours_str)  # Clean the working hours string
        else:
            working_hours_str = clean_string(str(working_hours))  # Clean the string representation
        
        sheet.cell(row=row, column=first_empty_column + 3, value=working_hours_str)  # New Working Hours
        sheet.cell(row=row, column=first_empty_column + 4, value=clean_string(record.get('contact_information', {}).get('primary_phone', '')))  # New Phone Number
        sheet.cell(row=row, column=first_empty_column + 5, value=clean_string(record.get('contact_information', {}).get('email', '')))  # New Email
        

        primary_services = record.get('services', {}).get('primary_services', [])
        if isinstance(primary_services, list):
            primary_services_cleaned = [clean_string(service) for service in primary_services]
            sheet.cell(row=row, column=first_empty_column + 6, value=", ".join(primary_services_cleaned))  # New Services
        else:
            sheet.cell(row=row, column=first_empty_column + 6, value=clean_string(primary_services))  # New Services

        secondary_services = record.get('services', {}).get('secondary_services', [])
        if isinstance(secondary_services, list):
            secondary_services_cleaned = [clean_string(service) for service in primary_services]
            sheet.cell(row=row, column=first_empty_column + 6, value=", ".join(secondary_services_cleaned))  # New Services
        else:
            sheet.cell(row=row, column=first_empty_column + 6, value=clean_string(secondary_services))  # New Services

# Save the updated Excel file
wb.save('updated_file_with_new_data.xlsx')


In [68]:
for record in data:
    print(record.get('Main_data_Row'))

0
2
5
14
17
21
29
31
33
34
35
39
40
45
47
48
58
60
63
73
74
76
77
81
83
85
87
88
90
91
92
98
99
102
103
105
106
109
113


In [59]:
pd.read_excel("updated_file_with_new_data.xlsx").iloc[:, 9:]

,New Business Name,New Address,New Rating,New Working Hours,New Phone Number,New Email,New Services
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
111,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,NaN,NaN,NaN,NaN,NaN,NaN,NaN
